# Notebook for Programming Question 1
Welcome to the programming portion of the assignment! you will be using [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true), so if you have never used it before, take a quick look through this introduction: [Working with Google Colab](https://docs.google.com/document/d/1LlnXoOblXwW3YX-0yG_5seTXJsb3kRdMMRYqs8Qqum4/edit?usp=sharing).

you be also be programming in Python, which we will assume a basic familiarity with. Python has fantastic community support and we'll be using numerous packages for machine learning (ML) and natural language processing (NLP) tasks.

### Learning Objectives
In this problem we will experiment with language models and implement smoothing. We will also see effects of using unigram/bigram LMs and the size of the training data.

### Writing Code
Look for the keyword "TODO" and fill in your code in the empty space.
Feel free to add and delete arguments in function signatures, but be careful that you might need to change them in function calls which are already present in the notebook.

### Data preprocessing

In this section we will write code to load data and clean (tokenize) it.

In [6]:
# Import libraries for preprocessing
import os
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
# Tokenizer class
# Fill all function blocks marked as TODO

class Tokenizer:
  def __init__(self, tokenize_type='basic', lowercase=False):
    self.lowercase = lowercase  # If this is True, convert text to lowercase while tokenizing.
    self.type = tokenize_type
    self.vocab = []


  """This simple tokenizer splits the text using whitespace."""
  def basicTokenize(self, string):
    words = string.split()
    return words

  ### TODO : Fill in this function to use NLTK's word_tokenize() function. ###
  def nltkTokenize(self, string):
    ##### SOLUTION START #####
    return nltk.word_tokenize(string)
    ##### SOLUTION END #####
    
  def tokenize(self, string):
    if self.lowercase:
      string = string.lower()
    if self.type == 'basic':
      tokens = self.basicTokenize(string)
    elif self.type == 'nltk':
      tokens = self.nltkTokenize(string)
    else:
      raise ValueError('Unknown tokenization type.')


    # Populate vocabulary
    self.vocab += [w for w in set(tokens) if w not in self.vocab]

    return tokens

  ### TODO: Fill in this function to return the top k words (and their frequency) in the corpus according to frequency. ###
  def countTopWords(self, words, k):
    ##### SOLUTION START #####
    fdist = nltk.FreqDist(word.lower() for word in self.tokenize(words))
    fdist.most_common(k)
    ##### SOLUTION END #####

In [2]:
# Function for reading the corpus
def readCorpus(filename, tokenizer):
  with open(filename) as f:
    words = tokenizer.tokenize(f.read())
  return words

### Language Modeling and Smoothing
In this section we will first compute the bigram counts and estimate bigram probabilities. We will then implement add-alpha smoothing to modify the probabilities.

In [3]:
# Import libraries
# Feel free to import as many as you like
import sys
import os
import numpy as np
import argparse
from tqdm import tqdm
from collections import Counter

In [6]:
# Class definition for language modeling
# Fill all function blocks marked as TODO

class LanguageModel:
  def __init__(self, vocab, n=2, smoothing=None, smoothing_param=None):
    assert n >=2, "This code does not allow you to train unigram models."
    self.vocab = vocab
    self.token_to_idx = {word: i for i, word in enumerate(self.vocab)}
    self.n = n
    self.smoothing = smoothing
    self.smoothing_param = smoothing_param
    self.bi_counts = None      # Holds the bigram counts
    self.bi_prob = None        # Holds the computed bigram probabilities.

    assert smoothing is None or smoothing_param is not None, "Forgot to specify a smoothing parameter?"


  """Compute basic bigram probabilities (without any smoothing)"""
  def computeBigramProb(self):
    self.bi_prob = self.bi_counts.copy()

    for i, _ in enumerate(tqdm(self.bi_prob, desc="Estimating bigram probabilities")):
      cnt = np.sum(self.bi_prob[i])
      if cnt > 0:
        self.bi_prob[i] /= cnt

  ### TODO: complete ###
  def computeBigramProbAddAlpha(self, alpha=0.001):
    ##### SOLUTION START #####

    ##### SOLUTION END #####
    return


  """Train a basic n-gram language model"""
  def train(self, corpus):
    if self.n==2:
      self.bi_counts = np.zeros((len(self.vocab), len(self.vocab)), dtype=float)
    else:
      raise ValueError("Only bigram model has been implemented so far.")

    # Convert to token indices.
    corpus = [self.token_to_idx[w] for w in corpus]

    # Gather counts
    for i, idx in enumerate(tqdm(corpus[:1-self.n], desc="Counting")):
      self.bi_counts[idx][corpus[i+1]] += 1

    # Pre-compute the probabilities.
    if not self.smoothing:
      self.computeBigramProb()
    elif self.smoothing == 'addAlpha':
      self.computeBigramProbAddAlpha(self.smoothing_param)
    else:
      raise ValueError("Unknown smoothing type.")



  def test(self, corpus):
    logprob = 0.

    # Convert to token indices.
    corpus = [self.token_to_idx[w] for w in corpus]

    for i, idx in enumerate(tqdm(corpus[:1-self.n], desc="Evaluating")):
      logprob += np.log(self.bi_prob[idx, corpus[i+1]])

    # import pdb; pdb.set_trace()

    logprob /= len(corpus[:1-self.n])

    # Compute perplexity
    ppl = np.exp(-logprob)

    return ppl

### Instantiate a tokenizer and LM, and calculate perplexity
This section contains driver code for learning a language model and evaluating it on a train and dev corpus.

In [7]:
def runLanguageModel(train_corpus,
                     val_corpus,
                     train_fraction,
                     tokenizer,
                     smoothing_type=None,
                     smoothing_param='0.0'):

  # Print the top 10 words in the corpus.
  # IMPORTANT: complete the function within the tokenizer class in data.py first.
  print("Top 10 words: %s" %(tokenizer.countTopWords(train_corpus, k=10)))

  # Instantiate the language model.
  lm = LanguageModel(tokenizer.vocab, n=2, smoothing=smoothing_type, smoothing_param=smoothing_param)

  # Figure out index for a specific percentage of train corpus to use.
  train_idx = int(train_fraction * len(train_corpus))

  lm.train(train_corpus[:train_idx])

  train_ppl = lm.test(train_corpus[:train_idx])
  val_ppl = lm.test(val_corpus)

  print("Train perplexity: %f, Val Perplexity: %f" %(train_ppl, val_ppl))

  return [train_ppl, val_ppl]

### Load the data

In [8]:
# Load training data here
# First download the datasets

!wget https://github.com/Tsegaye-misikir/NLP-rug/blob/main/brown/brown-train.txt
!wget https://github.com/Tsegaye-misikir/NLP-rug/blob/main/brown/brown-test.txt

# Instantiate a basic tokenizer
basic_tokenizer = Tokenizer(tokenize_type='basic', lowercase=True)
nltk_tokenizer = Tokenizer(tokenize_type='nltk', lowercase=True)

# Read the corpus and store
train_corpus = readCorpus('brown-train.txt', basic_tokenizer)
val_corpus = readCorpus('brown-test.txt', basic_tokenizer)
train_corpus_nltk = readCorpus('brown-train.txt', nltk_tokenizer)
val_corpus_nltk = readCorpus('brown-test.txt', nltk_tokenizer)

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'brown-train.txt'

### Experiments

#### Plot the frequency of words
Code for sub-part (a)(b)

In [ ]:
# TODO: Code for plotting the frequency of words
##### SOLUTION START #####

##### SOLUTION END #####

#### Report the train and test perplexity after learning the language model
Code for sub-part (c)

In [ ]:
# Train and test perplexity
runLanguageModel(train_corpus, val_corpus, train_fraction=1.0, tokenizer=basic_tokenizer)

#### Add-alpha smoothing
Code for sub-part (d)

In [ ]:
# Example command
runLanguageModel(train_corpus, val_corpus, train_fraction=1.0, tokenizer=basic_tokenizer, smoothing_type='addAlpha', smoothing_param=0.01)

# TODO: Part (d)
##### SOLUTION START #####


##### SOLUTION END #####